In [7]:
# naver view 정보 csv, xls로 저장

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import math
import pandas as pd
import os

print("=" * 80)
print(" 네이버 VIEW - 저장할 내용을 목록으로 만들어서 xls , csv 형식으로 저장하기")
print("=" * 80)

query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
cnt = int(input('2.수집할 데이터는 몇 건입니까?: '))
page_cnt = math.ceil(cnt / 30)

f_dir = input('3.결과를 저장할 폴더이름을 입력해주세요(기본경로: c:\\py_temp\\) :')
if f_dir == '':
    f_dir = 'c:\\py_temp\\'

chrome_path = "chromedriver.exe" 
service = Service(chrome_path) 
driver = webdriver.Chrome(service=service)  

driver.get('http://www.naver.com')
time.sleep(2)

element = driver.find_element(By.ID, "query")
element.send_keys(query_txt)
element.submit()

driver.find_element(By.LINK_TEXT, "블로그").click()
time.sleep(2)



 네이버 VIEW - 저장할 내용을 목록으로 만들어서 xls , csv 형식으로 저장하기


1.크롤링할 키워드는 무엇입니까?:  여행
2.수집할 데이터는 몇 건입니까?:  100
3.결과를 저장할 폴더이름을 입력해주세요(기본경로: c:\py_temp\) : 


In [9]:
no2 = [ ]           # 게시글 번호 
title2 = [ ]        # 게시물 제목 
contents2 = [ ]     # 게시글 내용 
bdate2 = [ ]        # 작성 일자 
nick2 = [ ]         # 블로그 닉네임
writer2 =[ ]        # 작성자

# 자동 스크롤다운
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(3)

if page_cnt > 2 :
    i = 1
    while (i <= page_cnt):
        scroll_down(driver) 
        i += 1
        print('%s 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^' %i)

print("\n")

2 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^
3 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^
4 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^
5 페이지 정보를 추출하고 있으니 잠시만 기다려 주세요~~^^




In [11]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
view_list = soup.find('ul','lst_view _fe_view_infinite_scroll_append_target').find_all('li')

no = 1  

for i in view_list :

    try :
        all_title = i.find('div','title_area').find_all('a')
        title = all_title[5].get_text( )          # 게시물 제목
    except :
        continue
    else :
        no2.append(no)                            
        print('1.번호:',no)

        title2.append(title)                      
        print('2.제목:',title)
    
        all_title = i.find_all('a')
        writer = all_title[4].get_text( )          
        writer2.append(writer)                      
        print('3.작성자:',writer)
        
        contents = i.find('div' , 'dsc_link').get_text( )   
        contents2.append(contents)                
        print('4.요약내용:',contents)

        bdate = i.find('span','sub').get_text( )  
        bdate2.append(bdate)                     
        print('5.작성일자:',bdate)

        print("\n")

        if no == cnt :
            break

        no += 1


In [13]:
import openpyxl

naver_view = pd.DataFrame()
naver_view['번호'] = no2
naver_view['제목'] = pd.Series(title2)
naver_view['작성자'] =pd.Series(writer2)
naver_view['요약내용'] =pd.Series(contents2)
naver_view['작성일자'] = pd.Series(bdate2)

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

sec_name = 'Naver_View'
os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xlsx'

naver_view.to_csv(fc_name,encoding="utf-8-sig",index=False)
print(" csv 파일 저장 경로: %s" %fc_name) 

naver_view.to_excel(fx_name , index=False , engine='openpyxl')
print(" xls 파일 저장 경로: %s" %fx_name) 

 csv 파일 저장 경로: c:\py_temp\2025-01-11-03-57-57-여행-Naver_View\2025-01-11-03-57-57-여행-Naver_View.csv
 xls 파일 저장 경로: c:\py_temp\2025-01-11-03-57-57-여행-Naver_View\2025-01-11-03-57-57-여행-Naver_View.xlsx
